In [14]:
#Input & Output file Names

#Input Files
base_file = r'D:/My Projects/VS Code/Test codes/IPL2024_Match_information.xlsx'  #match_information  from link scrap code
date_time_file = r'D:\My Projects\VS Code\Test codes\IPL2024_Match_Date&Time details.xlsx'
match_innings_details = r'D:\My Projects\VS Code\Test codes\IPL2024_Innings_Details.xlsx'

#Output files
match_information_file = r'D:/My Projects/VS Code/Test codes/Final_IPL2024_Match_information_updated.xlsx'
match_date_time = r'D:\My Projects\VS Code\Test codes\Final_IPL2024_Match_Date&Time_Details.xlsx'

innings_details = r"D:\My Projects\VS Code\Test codes\IPL2024_Innings_Details_Modified_Innings.xlsx"

indian_premier_league = "Indian Premier League 2024"
batting_innings = r"D:\My Projects\VS Code\Test codes\Final_IPL2024_Batting_Innings_Details.xlsx"
bowling_innings = r"D:\My Projects\VS Code\Test codes\Final_IPL2024_Bowling_Innings_Details.xlsx"


MATCH INFORMATION

In [15]:
import pandas as pd


file_path = base_file
df = pd.read_excel(file_path)


df[['Match', 'Match No', 'League']] = df['Match'].str.split(', ', expand=True, n=2)


df[['Umpire1', 'Umpire2']] = df['Umpires'].str.split(', ', expand=True, n=1)


df = df.drop(columns=['Date', 'Time', 'Umpires', 'Team', 'Role', 'Players'])


team_mapping = {
    'CSK': 'Chennai Super Kings',
    'RCB': 'Royal Challengers Bengaluru',
    'PBKS': 'Punjab Kings',
    'DC': 'Delhi Capitals',
    'KKR': 'Kolkata Knight Riders',
    'SRH': 'Sunrisers Hyderabad',
    'RR': 'Rajasthan Royals',
    'LSG': 'Lucknow Super Giants',
    'GT': 'Gujarat Titans',
    'MI': 'Mumbai Indians'
}


df[['Team 1', 'Team 2']] = df['Match'].str.split(' vs ', expand=True)


df['Team 1'] = df['Team 1'].map(team_mapping)
df['Team 2'] = df['Team 2'].map(team_mapping)


df = df.drop_duplicates()


df = df[['Match No', 'League', 'Match','Team 1', 'Team 2', 'Venue', 'Toss', 'Umpire1', 'Umpire2', 'Third Umpire', 'Match Referee' ]]


new_file_path = match_information_file
df.to_excel(new_file_path, index=False)

print("Updated file with duplicate values removed and Team 1, Team 2 columns saved successfully!")


Updated file with duplicate values removed and Team 1, Team 2 columns saved successfully!


Match DATE & TIME UPDATE FILE

In [16]:
import pandas as pd


file_path = date_time_file
df = pd.read_excel(file_path)


df[['Match', 'Match No']] = df['Match'].str.split(',', n=1, expand=True)
df['Match No'] = df['Match No'].str.split('-').str[0].str.strip()
df[['Team1', 'Team2']] = df['Match'].str.split(' vs ', expand=True)


df['Year'] = df['Series'].str.extract(r'(\d{4})')
df[['Date', 'Time']] = df['Date & Time'].str.split(',', n=1, expand=True)
df['Date'] = df['Date'].str.strip() + ', ' + df['Year']


df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y').dt.date


df['Time'] = df['Time'].str.replace('LOCAL', '').str.strip()
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M %p').dt.strftime('%I:%M %p')


output_df = df[['Match No', 'Match', 'Team1', 'Team2', 'Series', 'Venue', 'Date', 'Time']]


output_file_path = match_date_time
output_df.to_excel(output_file_path, index=False)

print("Successfully created")


Successfully created


INNINGS SPLIT DETAILS

In [17]:
import pandas as pd

file_path = match_innings_details
output_file_path = innings_details 

df = pd.read_excel(file_path)

df[['Match', 'Match No']] = df['Match Title'].str.split(', ', n=1, expand=True)

df['Match No'] = df['Match No'].str.split(' - ').str[0]

df['Team1'] = df['Match'].str.split(' vs ').str[0]
df['Team2'] = df['Match'].str.split(' vs ').str[1]

def process_innings_details(group):
    innings_data = {
        "1st Batting Innings": "",
        "1st Bowling Innings": "",
        "1st Batting Power Play": "",
        "2nd Batting Innings": "",
        "2nd Bowling Innings": "",
        "2nd Batting Power Play": ""
    }
    if len(group) >= 1:
        innings_data["1st Batting Innings"] = group.iloc[0]['Innings Details']
    if len(group) >= 2:
        innings_data["1st Bowling Innings"] = group.iloc[1]['Innings Details']
    if len(group) >= 3:
        innings_data["1st Batting Power Play"] = group.iloc[2]['Innings Details']
    if len(group) >= 4:
        innings_data["2nd Batting Innings"] = group.iloc[3]['Innings Details']
    if len(group) >= 5:
        innings_data["2nd Bowling Innings"] = group.iloc[4]['Innings Details']
    if len(group) >= 6:
        innings_data["2nd Batting Power Play"] = group.iloc[5]['Innings Details']
    return pd.Series(innings_data)

df_innings = df.groupby("Match No").apply(process_innings_details).reset_index()

df_output = df.drop(columns=['Innings Details']).drop_duplicates('Match No').merge(df_innings, on="Match No")

df_output = df_output[['Match', 'Match No', 'Team1', 'Team2', 
                       '1st Batting Innings', '1st Bowling Innings', 
                       '1st Batting Power Play', '2nd Batting Innings', 
                       '2nd Bowling Innings', '2nd Batting Power Play']]

df_output.to_excel(output_file_path, index=False)
print("Successfully created")


Successfully created


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9120\1479545118.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_innings = df.groupby("Match No").apply(process_innings_details).reset_index()


BATTING SCORECARD

In [18]:
import pandas as pd
import re

file_path = innings_details

df = pd.read_excel(file_path)

def parse_batting_innings(cell_data, match_no, match):
    score_pattern = r"(.+?Innings \d+-\d+ \((?:[0-9]|1[0-9]|20)(?:\.\d)? Ov\))"
    player_pattern = r"\b(?:6s\s*SR\s+)?([a-zA-Z\s\(\)&]+?)(?=\s{2,})(?:\s{2,})(c\s\(sub\)?[a-zA-Z\s]+b\s[a-zA-Z\s]+|c\s[a-zA-Z\s]+b\s[a-zA-Z\s]+|run out\s\([a-zA-Z\s]+\)|b\s[a-zA-Z\s]+|not out|Did not Bat)\s*(\d*)\s*(\d*)\s*(\d*)\s*(\d*)\s*(\d*\.\d*)"

    team_score_match = re.search(score_pattern, cell_data)
    team_score = team_score_match.group(1) if team_score_match else ""

    team_info_pattern = r"(.+?)\s+(\d+)-(\d+)\s+\((\d+(?:\.\d+)?) Ov\)"
    team_info_match = re.search(team_info_pattern, team_score)
    
    if team_info_match:
        batting_team = team_info_match.group(1).strip()
        total_score = team_info_match.group(2).strip()
        wickets = team_info_match.group(3).strip()
        overs = team_info_match.group(4).strip()
    else:
        batting_team = total_score = wickets = overs = ""

    players = re.findall(player_pattern, cell_data)

    parsed_data = []
    
    extras_pattern = r"Extras\s+(\d+)"
    extras_match = re.search(extras_pattern, cell_data)
    extras_runs = extras_match.group(1) if extras_match else "0"

    parsed_data.append({
        "Match No": match_no,
        "Match": match,
        "Batting Team": batting_team,
        "Total Score": extras_runs,
        "Wickets": "0",  
        "Overs": "0",     
        "Batter": "Extras",
        "Dismissal": "",  
        "Runs": extras_runs,
        "Balls": "0",     
        "4s": "0",        
        "6s": "0",
        "SR": "0",
    })

    for player in players:
        batter, dismissal, runs, balls, fours, sixes, sr = player
        parsed_data.append({
            "Match No": match_no,
            "Match": match,
            "Batting Team": batting_team,
            "Total Score": total_score,
            "Wickets": wickets,
            "Overs": overs,
            "Batter": batter.strip(),
            "Dismissal": dismissal.strip(),
            "Runs": runs.strip(),
            "Balls": balls.strip(),
            "4s": fours.strip(),
            "6s": sixes.strip(),
            "SR": sr.strip()
        })

    return parsed_data

# 1st Batting Innings
parsed_rows_1st = []
for index, cell in df['1st Batting Innings'].dropna().items():
    match_no = df.at[index, 'Match No']
    match = df.at[index, 'Match']
    parsed_rows_1st.extend(parse_batting_innings(cell, match_no, match))

output_df_1st = pd.DataFrame(parsed_rows_1st)

numeric_columns = ["Total Score", "Wickets", "Overs", "Runs", "Balls", "4s", "6s", "SR"]
for col in numeric_columns:
    output_df_1st[col] = pd.to_numeric(output_df_1st[col], errors='coerce')

output_df_1st["Batting Innings"] = "1st Innings"
output_df_1st["Series"] = indian_premier_league

# 2nd Batting Innings
parsed_rows_2nd = []
for index, cell in df['2nd Batting Innings'].dropna().items():
    match_no = df.at[index, 'Match No']
    match = df.at[index, 'Match']
    parsed_rows_2nd.extend(parse_batting_innings(cell, match_no, match))

output_df_2nd = pd.DataFrame(parsed_rows_2nd)

for col in numeric_columns:
    output_df_2nd[col] = pd.to_numeric(output_df_2nd[col], errors='coerce')

output_df_2nd["Batting Innings"] = "2nd Innings"
output_df_2nd["Series"] = indian_premier_league

combined_output_df = pd.concat([output_df_1st, output_df_2nd], ignore_index=True)

output_file_path = batting_innings
combined_output_df.to_excel(output_file_path, sheet_name='Batting Innings', index=False)

print("Successfully created")


Successfully created


BOWLING SCORECARD

In [19]:
import pandas as pd
import re

file_path = innings_details

df = pd.read_excel(file_path)

def parse_bowling_innings(cell_data, match_no, match):
    bowler_pattern = r"(?:Bowler\s+O\s+M\s+R\s+W\s+NB\s+WD\s+ECO\s+)?([a-zA-Z\s\(\)]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+\.\d+)"
    
    bowlers = re.findall(bowler_pattern, cell_data)
    
    parsed_data = []
    for bowler in bowlers:
        name, overs, maiden, runs, wickets, nb, wd, eco = bowler
        parsed_data.append({
            "Match No": match_no,
            "Match": match,
            "Bowler": name.strip(),
            "Over": overs.strip(),
            "Maiden": maiden.strip(),
            "Runs": runs.strip(),
            "Wickets": wickets.strip(),
            "NB": nb.strip(),
            "WD": wd.strip(),
            "ECO": eco.strip()
        })
    
    return parsed_data

# 1st Bowling Innings
parsed_rows_1st_bowling = []
for index, cell in df['1st Bowling Innings'].dropna().items():
    match_no = df.at[index, 'Match No']
    match = df.at[index, 'Match']
    parsed_rows_1st_bowling.extend(parse_bowling_innings(cell, match_no, match))

output_df_1st_bowling = pd.DataFrame(parsed_rows_1st_bowling)
output_df_1st_bowling["Bowling Innings"] = "1st Innings"

# 2nd Bowling Innings
parsed_rows_2nd_bowling = []
for index, cell in df['2nd Bowling Innings'].dropna().items():
    match_no = df.at[index, 'Match No']
    match = df.at[index, 'Match']
    parsed_rows_2nd_bowling.extend(parse_bowling_innings(cell, match_no, match))

output_df_2nd_bowling = pd.DataFrame(parsed_rows_2nd_bowling)
output_df_2nd_bowling["Bowling Innings"] = "2nd Innings"

combined_bowling_df = pd.concat([output_df_1st_bowling, output_df_2nd_bowling], ignore_index=True)

# Add the "Series" column with constant data "Indian Premier League 2024"
combined_bowling_df["Series"] = indian_premier_league

numeric_columns = ["Over", "Maiden", "Runs", "Wickets", "NB", "WD", "ECO"]
for col in numeric_columns:
    combined_bowling_df[col] = pd.to_numeric(combined_bowling_df[col], errors='coerce')

output_file_path = bowling_innings
combined_bowling_df.to_excel(output_file_path, sheet_name='Bowling Innings', index=False)

print("Successfully created")


Successfully created
